# Understand the kernel with code samples

----
The kernel is the central component of Semantic Kernel. At its simplest, the kernel is a Dependency Injection container that manages all of the services and plugins necessary to run your AI application. If you provide all of your services and plugins to the kernel, they will then be seamlessly used by the AI as needed. 

| Components | Description |
|------------|-------------|
| **1. Services** | These consist of both AI services (e.g., chat completion) and other services (e.g., logging and HTTP clients) that are necessary to run your application. This was modeled after the Service Provider pattern in .NET to support dependency injection across all languages. |
| **2. Plugins** | These are components used by your AI services and prompt templates to perform work. For example, AI services can use plugins to retrieve data from a database or call an external API to perform actions. |

![kernel](images/the-kernel-is-at-the-center-of-everything.png)



## Hierarchical Architecture Diagram
---

Architecture of a Retrieval-Augmented Generation (RAG) based chatbot using the latest Semantic Kernel. The diagram highlights how the Agent Framework (for chat interactions) and Process Framework (for workflow orchestration) sit above the core Kernel, leveraging it to perform AI tasks. The Semantic Kernel core orchestrates between user requests and the underlying AI services, memory stores, and plugins. It integrates with external LLM services for generation, uses a vector DB for semantic memory (retrieval of relevant data), and can invoke plugins which may call other external APIs or services as needed.

![Hierarchical](images/semantic_kernel_component.png)

***AI Agent (Agent Framework)***: The Agent Framework in Semantic Kernel is an optional layer that helps create conversational AI agents (like chatbots) using the core kernel’s capabilities​. It is not a replacement for the kernel but builds on top of it – your application still includes the Semantic Kernel library, and the agent uses the kernel’s functions internally. In a chatbot scenario, the Agent Framework manages the dialogue (turn-taking, system prompts, etc.) while delegating AI tasks to the kernel.

***Semantic Kernel (Core)***: The Semantic Kernel core is the heart of the system. It orchestrates calls to AI models, retrieves memories, and executes plugin functions. The kernel provides abstractions to connect to AI services (LLM APIs for text generation, embedding models, etc.) and to various memory stores (for example, vector databases)​. It also manages context (prompts) and can use Planners to chain or select functions dynamically to fulfill a user request​. The kernel itself is part of your app’s runtime, coordinating all other components.

***Plugins (Skills/Functions)***: Plugins (also called skills or functions) are units of functionality that the kernel can invoke. They might be defined with natural language prompts (semantic functions) or as native code functions. Plugins can perform calculations, transform data, or call external services/APIs. They are registered with and executed via the kernel, meaning they depend on the kernel to be invoked as part of an AI workflow​. However, the plugin implementations (e.g. an HTTP call to a web service, a database query) run outside the kernel – the kernel just orchestrates their usage. In essence, plugins extend the kernel’s abilities, and the kernel can automatically chain plugins to accomplish complex tasks for the user​.

***AI Services***: These are external AI model endpoints that the kernel calls through its connectors. For example, the OpenAI or Azure OpenAI service provides the GPT-4 model for text generation, and there are embedding model services for vector generation. The kernel has integrations for many AI services (text completion, chat, image generation, speech recognition, etc.) and it uses them by calling out to the respective APIs​. These services are not “inside” the kernel – instead, the kernel depends on them to provide the intelligence. In the architecture, they appear as external components that the kernel invokes (e.g. sending a prompt to the GPT model and getting a completion).

***Semantic Memory (Vector DB)***: The Semantic Kernel includes a memory abstraction that allows storing and retrieving contextual information. Under the hood, this is often backed by a vector database or search index. In a RAG-based chatbot, this is critical: documents or knowledge are embedded into vector representations and stored, so that relevant pieces can be retrieved to ground the AI’s answers. Semantic Kernel’s memory can integrate with many vector stores (e.g. Azure Cognitive Search, ChromaDB, Qdrant, Redis, Pinecone, etc.)​. This means “Memory” is essentially an AI Search over a vector DB, enabling the bot to find relevant information by semantic similarity. The memory component is used by the kernel but the database itself is external – the kernel just sends queries and gets results.

***Process Framework (Workflow Orchestration)***: The Process Framework is another optional layer in the latest Semantic Kernel, aimed at long-running or multi-step workflows. It lets developers define structured processes composed of multiple steps, where each step can call kernel functions (AI or non-AI tasks) in an event-driven sequence​. Like the Agent Framework, the Process Framework builds on the kernel rather than enclosing it. It uses the kernel to execute AI functions at each step of a business workflow. This is especially useful if your chatbot or assistant needs to carry out complex transactions or procedures (for example, an order processing workflow or a support ticket resolution that involves several back-and-forth steps) beyond a single conversational turn. The Process Framework uses technologies like Orleans or Dapr under the hood for reliability and can reuse any existing kernel plugins within those processes​.

# Setup

Let's define our kernel for this example.

In [13]:
import asyncio
from typing import Annotated
from dotenv import load_dotenv
import json
import os
from enum import Enum
from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import AzureChatPromptExecutionSettings
from semantic_kernel.prompt_template import PromptTemplateConfig
from semantic_kernel.agents import AssistantAgentThread, AzureAssistantAgent
from semantic_kernel.kernel_pydantic import KernelBaseSettings
from semantic_kernel.connectors.search.bing import BingSearch
from semantic_kernel.functions import KernelArguments, KernelParameterMetadata, KernelPlugin
from semantic_kernel.connectors.ai import FunctionChoiceBehavior
from semantic_kernel.contents import ChatHistory
from semantic_kernel.filters import FilterTypes, FunctionInvocationContext
from collections.abc import Awaitable, Callable
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentSettings, AzureAIAgentThread
from semantic_kernel.agents import AzureResponsesAgent, ResponsesAgentThread
from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.functions import kernel_function
from semantic_kernel.agents import AzureResponsesAgent
from azure.identity.aio import DefaultAzureCredential
from semantic_kernel.agents.strategies.termination.termination_strategy import TerminationStrategy
from semantic_kernel.agents import AgentGroupChat
from semantic_kernel.contents.utils.author_role import AuthorRole
from azure.ai.projects.models import OpenApiAnonymousAuthDetails, OpenApiTool, CodeInterpreterTool
from semantic_kernel.contents import AuthorRole
from semantic_kernel.contents.streaming_chat_message_content import StreamingChatMessageContent


kernel = Kernel()



class Service(Enum):
    """Attributes:
    OpenAI (str): Represents the OpenAI service.
    AzureOpenAI (str): Represents the Azure OpenAI service.
    HuggingFace (str): Represents the HuggingFace service.
    """

    OpenAI = "openai"
    AzureOpenAI = "azureopenai"
    HuggingFace = "huggingface"

class ServiceSettings(KernelBaseSettings):
    """The Learn Resources Service Settings.

    The settings are first loaded from environment variables. If the
    environment variables are not found, the settings can be loaded from a .env file with the
    encoding 'utf-8' as default or the specific encoding. If the settings are not found in the
    .env file, the settings are ignored; however, validation will fail alerting that the settings
    are missing.

    Args:
        global_llm_service (str | None): The LLM service to use for the samples, either "OpenAI" or "AzureOpenAI"
            If not provided, defaults to "AzureOpenAI".
    """

    global_llm_service: str | None = None
    
load_dotenv(override=True)

True

We will load our settings and get the LLM service to use for the notebook.

In [14]:
service_settings = ServiceSettings.create()

# Select a service to use for this notebook (available services: OpenAI, AzureOpenAI, HuggingFace)
selectedService = (
    Service.AzureOpenAI
    if service_settings.global_llm_service is None
    else Service(service_settings.global_llm_service.lower())
)
print(f"Using service type: {selectedService}")

Using service type: Service.AzureOpenAI


We now configure our Chat Completion service on the kernel.

In [15]:
# Remove all services so that this cell can be re-run without restarting the kernel
kernel.remove_all_services()

service_id = None
if selectedService == Service.OpenAI:
    from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion

    service_id = "default"
    kernel.add_service(
        OpenAIChatCompletion(
            service_id=service_id,
        ),
    )
elif selectedService == Service.AzureOpenAI:
    from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

    service_id = "default"
    kernel.add_service(
        AzureChatCompletion(
            service_id=service_id,
        ),
    )

# Run a Semantic Function (No Agent, No Process)


## 🧪 Case 1.1 running a prompt without plugin
---

In [4]:


# Define the request settings
req_settings = kernel.get_prompt_execution_settings_from_service_id(service_id)
req_settings.max_tokens = 2000
req_settings.temperature = 0.7
req_settings.top_p = 0.8

prompt="tell me about the history of {{$company_name}} as TLDR in exactly 200 words. make line seperated by commas. \n\nTLDR:"

prompt_template_config = PromptTemplateConfig(
    template=prompt,
    name="tldr",
    template_format="semantic-kernel",
    execution_settings=req_settings,
)

function = kernel.add_function(
    function_name="tldr_function",
    plugin_name="tldr_plugin",
    prompt_template_config=prompt_template_config,
)


########################################
# invoke mode 
########################################

# result = await kernel.invoke(function)
# print(result) # => Robots must not harm humans.

########################################
# invoke_stream mode 
########################################

response = kernel.invoke_stream(plugin_name="tldr_plugin", function_name="tldr_function", company_name="Microsoft")
all_chunks_str = ""
print("Assistant:> ", end="")
async for chunk in response:
    if isinstance(chunk[0], StreamingChatMessageContent) and chunk[0].role == AuthorRole.ASSISTANT:
        all_chunks_str += str(chunk[0])
        print(str(chunk[0]), end="")
            
            

Assistant:> 

Microsoft was founded by Bill Gates and Paul Allen in 1975,  
Initially developed a version of BASIC for the Altair 8800,  
In 1980, Microsoft signed a contract with IBM to provide an operating system,  
This led to the creation of MS-DOS, which became the industry standard,  
The introduction of Windows in 1985 marked a significant shift towards graphical user interfaces,  
By the 1990s, Windows became the dominant OS for personal computers,  
Microsoft Office suite was launched, enhancing productivity for businesses,  
The company went public in 1986, making Gates one of the youngest billionaires,  
Antitrust scrutiny began in the late 1990s, resulting in legal battles,  
In 2001, Microsoft launched Windows XP, which became widely popular,  
The company expanded into hardware with the Xbox in 2001,  
In the 2010s, Microsoft shifted focus towards cloud computing and services,  
The acquisition of LinkedIn in 2016 marked a significant investment in social networking,  
Satya Nadella be

## 🧪 Case 1.2 running a prompt with plugin
---

In [5]:
plugin = kernel.add_plugin(parent_directory="prompt_template_samples/", plugin_name="FunPlugin")

In [6]:
from semantic_kernel.functions import KernelArguments

joke_function = plugin["Joke"]

joke = await kernel.invoke(
    joke_function,
    KernelArguments(input="time travel to dinosaur age", style="super silly"),
)
print(joke)

Why did the time traveler bring a ladder to the dinosaur age?

Because they heard the T-Rex was a little "short-tempered" and wanted to reach new heights in their friendship!


## 🧪 Case 1.3 running a prompt with web search plugin
---

In [7]:
import semantic_kernel.connectors.search.bing
kernel = Kernel()
kernel.add_service(AzureChatCompletion(service_id="chat"))
bing_search_plugin = kernel.add_plugin(KernelPlugin.from_text_search_with_search(
        semantic_kernel.connectors.search.bing.BingSearch(),
        plugin_name="bing",
        description="Search the web information using bing search.",
        parameters=[
            KernelParameterMetadata(
                name="query",
                description="The search query.",
                type="str",
                is_required=True,
                type_object=str,
            ),
            KernelParameterMetadata(
                name="top",
                description="The number of results to return.",
                type="int",
                is_required=False,
                default_value=2,
                type_object=int,
            ),
            KernelParameterMetadata(
                name="skip",
                description="The number of results to skip.",
                type="int",
                is_required=False,
                default_value=0,
                type_object=int,
            ),
            KernelParameterMetadata(
                name="site",
                description="The site to search.",
                default_value="",
                type="str",
                is_required=False,
                type_object=str,
            ),
        ],
    ))

chat_function = kernel.add_function(
    prompt="{{$chat_history}}{{$user_input}}",
    plugin_name="ChatBot",
    function_name="Chat",
)
execution_settings = AzureChatPromptExecutionSettings(
    service_id="chat",
    max_tokens=2000,
    temperature=0.7,
    top_p=0.8,
    function_choice_behavior=FunctionChoiceBehavior.Auto(auto_invoke=True),
)

history = ChatHistory()
system_message = """
You are a chat bot, use bing plugin to find answers.
"""
history.add_system_message(system_message)
history.add_user_message("Hi there, who are you?")
history.add_assistant_message("I am Information Finder, a chat bot. use bing plugin to find answers.")

arguments = KernelArguments(settings=execution_settings)

@kernel.filter(filter_type=FilterTypes.FUNCTION_INVOCATION)
async def log_bing_filter(
    context: FunctionInvocationContext, next: Callable[[FunctionInvocationContext], Awaitable[None]]
):
    if context.function.plugin_name == "bing":
        print("Calling Bing search with arguments:")
        if "query" in context.arguments:
            print(f'  Query: "{context.arguments["query"]}"')
        if "count" in context.arguments:
            print(f'  Count: "{context.arguments["count"]}"')
        if "skip" in context.arguments:
            print(f'  Skip: "{context.arguments["skip"]}"')
        await next(context)
        print("Bing search completed.")
    else:
        await next(context)


async def chat() -> bool:
    try:
        user_input = input("User:> ")
    except KeyboardInterrupt:
        print("\n\nExiting chat...")
        return False
    except EOFError:
        print("\n\nExiting chat...")
        return False

    if user_input == "exit":
        print("\n\nExiting chat...")
        return False
    arguments["user_input"] = user_input
    arguments["chat_history"] = history
    result = await kernel.invoke(chat_function, arguments=arguments)
    print(f"SearchChatbot:> {result}")
    history.add_user_message(user_input)
    history.add_assistant_message(str(result))
    return True



chatting = True
print(
    "Welcome to the chat bot!\
    \n  Type 'exit' to exit.\
    \n  Try to search weather, news and more using bing search tool."
)
while chatting:
    chatting = await chat()

Welcome to the chat bot!    
  Type 'exit' to exit.    
  Try to search weather, news and more using bing search tool.
SearchChatbot:> How can I assist you today? If you have any questions or need information, feel free to ask!
Calling Bing search with arguments:
  Query: "current weather in South Korea"
Bing search completed.
SearchChatbot:> The current weather in South Korea, specifically in Seoul, is approximately 46°F and mostly cloudy. You can expect light showers and overcast conditions throughout the day. If you need more specific information or forecasts for other cities in South Korea, just let me know!


Exiting chat...


In [8]:
from semantic_kernel.functions import KernelArguments

bing_search_fuction = kernel.get_function("bing", "search")

search_result = await kernel.invoke(
    bing_search_fuction,
    KernelArguments(
        query="what is new opening hotels in NYC 2025?",
        top=10,
        skip=0,
    ),
)
print(search_result)

Calling Bing search with arguments:
  Query: "what is new opening hotels in NYC 2025?"
  Skip: "0"


Bing search completed.
Luxury Hilton Hotel in NYC reopening in 2025. This historic luxury hotel in the Midtown Manhattan area of New York City is currently undergoing a transformative restoration and is due to reopen on September 1, 2025 (the opening date may change). Read Reviews / Check Prices. New Hotel in SoHo NYC opened in September 2024.,New York is poised to welcome new hotels in New York City and beyond in 2025. Add these five to your must-stay list. Faena New York. Expected to debut in spring 2025, Faena New York will offer visitors stunning views of the High Line and Hudson River from One High Line’s East Tower.,New Hotels in Manhattan NY 2025! Be the first to sleep on new sheets, dine at buzz-worthy restaurants, sip cocktails at the hottest rooftops.,From the reimagining of a legendary hotel in New York to breathtaking, newly-built resorts across virtually untouched landscapes, this is the year of monumental hotel openings.,We have created a list with the best newly opened h

# Use single agent to chat


In [9]:
###################################################################
# The following sample demonstrates how to create a simple,       #
# Azure AI agent, ChatCompletionAgent, AzureResponsesAgent        #
# that answers questions about a sample menu                      #
# using a Semantic Kernel Plugin.                                 #
###################################################################


# Define a sample plugin for the sample
class MenuPlugin:
    """A sample Menu Plugin used for the concept sample."""

    @kernel_function(description="Provides a list of specials from the menu.")
    def get_specials(self) -> Annotated[str, "Returns the specials from the menu."]:
        return """
        Special Soup: Clam Chowder
        Special Salad: Cobb Salad
        Special Drink: Chai Tea
        """

    @kernel_function(description="Provides the price of the requested menu item.")
    def get_item_price(
        self, menu_item: Annotated[str, "The name of the menu item."]
    ) -> Annotated[str, "Returns the price of the menu item."]:
        return "$9.99"


# Simulate a conversation with the agent
USER_INPUTS = [
    "Hello",
    "What is the special soup?",
    "How much does that cost?",
    "Thank you",
]

## 🧪 Case 2.1 chat with Azure AI Agent 
---

In [10]:
# Make sure to set the environment variables for the Azure AI Agent
ai_agent_settings = AzureAIAgentSettings.create()

async with (
    DefaultAzureCredential() as creds,
    AzureAIAgent.create_client(credential=creds) as client,
):
    # 1. Create an agent on the Azure AI agent service
    agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        name="Host",
        instructions="Answer questions about the menu.",
    )

    # 2. Create a Semantic Kernel agent for the Azure AI agent
    agent = AzureAIAgent(
        client=client,
        definition=agent_definition,
        plugins=[MenuPlugin()],  # Add the plugin to the agent
    )

    # 3. Create a thread for the agent
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: AzureAIAgentThread = None

########################################
# invoke mode 
########################################

    # for user_input in USER_INPUTS:
    #     print(f"# User: {user_input}")
    #     # 4. Invoke the agent for the specified thread for response
    #     async for response in agent.invoke(
    #         messages=user_input,
    #         thread=thread,
    #     ):
    #         print(f"# {response.name}: {response}")
    #         thread = response.thread
    
    # await thread.delete() if thread else None    

########################################
# invoke_stream mode 
########################################
    try:
        for user_input in USER_INPUTS:
            print(f"# User: '{user_input}'")
            first_chunk = True
            # 4. Invoke the agent for the current message and print the response
            async for response in agent.invoke_stream(messages=user_input, thread=thread):
                thread = response.thread
                if first_chunk:
                    print(f"# {response.name}: ", end="", flush=True)
                    first_chunk = False
                print(response.content, end="", flush=True)
            print()
    finally:
            # Cleanup: Delete the thread and agent
            await thread.delete() if thread else None
            await client.agents.delete_agent(agent.id)
        

# User: 'Hello'
# Host: Hello! How can I assist you today?
# User: 'What is the special soup?'
# Host: The special soup is Clam Chowder. Is there anything else you'd like to know about the menu?
# User: 'How much does that cost?'
# Host: The Clam Chowder costs $9.99. If there's anything else you'd like to know, feel free to ask!
# User: 'Thank you'
# Host: You're welcome! If you have any more questions, feel free to ask. Enjoy your day!


## 🧪 Case 2.2 chat with ChatCompletionAgent
---

In [11]:
agent = ChatCompletionAgent(
        service=AzureChatCompletion(),
        name="Host",
        instructions="Answer questions based on web search",
        plugins=[MenuPlugin()],
    )

# 2. Create a thread to hold the conversation
# If no thread is provided, a new thread will be
# created and returned with the initial response
thread: ChatHistoryAgentThread = None

########################################
# invoke mode 
########################################

# for user_input in USER_INPUTS:
#     print(f"# User: {user_input}")
#     # 4. Invoke the agent for a response
#     response = await agent.get_response(messages=user_input, thread=thread)
#     print(f"# {response.name}: {response} ")
#     thread = response.thread

########################################
# invoke_stream mode 
########################################

for user_input in USER_INPUTS:
        print(f"# User: '{user_input}'")
        first_chunk = True
        # 4. Invoke the agent for the current message and print the response
        async for response in agent.invoke_stream(messages=user_input, thread=thread):
            thread = response.thread
            if first_chunk:
                print(f"# {response.name}: ", end="", flush=True)
                first_chunk = False
            print(response.content, end="", flush=True)
        print()


await thread.delete() if thread else None

# User: 'Hello'
# Host: Hello! How can I assist you today?


# User: 'What is the special soup?'
# Host: The special soup is Clam Chowder. Would you like to know more about it or anything else?
# User: 'How much does that cost?'
# Host: The Clam Chowder costs $9.99. Would you like to know more about the menu or anything else?
# User: 'Thank you'
# Host: You're welcome! If you have any more questions or need assistance, feel free to ask. Have a great day!


## 🧪 Case 2.3 chat with ResponseAgent (OpenAI Responses API)
---

In [16]:
client, model = AzureResponsesAgent.setup_resources()

# 2. Create a Semantic Kernel agent for the OpenAI Responses API
agent = AzureResponsesAgent(
    ai_model_id=model,
    client=client,
    instructions="Answer questions about the menu.",
    name="Host",
    plugins=[MenuPlugin()],
)

# 3. Create a thread for the agent
# If no thread is provided, a new thread will be
# created and returned with the initial response
thread: ResponsesAgentThread = None

########################################
# invoke mode 
########################################
# for user_input in USER_INPUTS:
#     print(f"# User: '{user_input}'")
#     # 4. Invoke the agent for the current message and print the response
#     response = await agent.get_response(messages=user_input, thread=thread)
#     print(f"# {response.name}: {response.content}")
#     thread = response.thread

########################################
# invoke_stream mode 
########################################
for user_input in USER_INPUTS:
        print(f"# User: '{user_input}'")
        first_chunk = True
        # 4. Invoke the agent for the current message and print the response
        async for response in agent.invoke_stream(messages=user_input, thread=thread):
            thread = response.thread
            if first_chunk:
                print(f"# {response.name}: ", end="", flush=True)
                first_chunk = False
            print(response.content, end="", flush=True)
        print()

await thread.delete() if thread else None

# User: 'Hello'


# Host: Hi there! How can I assist you today?
# User: 'What is the special soup?'
# Host: The special soup is Clam Chowder. Would you like to know more about it?
# User: 'How much does that cost?'
# Host: The Clam Chowder costs $9.99. Would you like to order anything else?
# User: 'Thank you'
# Host: You're welcome! If you have any more questions or need assistance, feel free to ask. Enjoy your day!


# Use tools to answer specific questions 

## 🧪 Case 3.1 code interpreter with Azure AI Agent 
---

In [17]:
TASK = "Use code to determine the values in the Fibonacci sequence that that are less then the value of 101."

ai_agent_settings = AzureAIAgentSettings.create()

async with (
    DefaultAzureCredential() as creds,
    AzureAIAgent.create_client(credential=creds) as client,
):
    # 1. Create an agent with a code interpreter on the Azure AI agent service
    code_interpreter = CodeInterpreterTool()
    agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        tools=code_interpreter.definitions,
        tool_resources=code_interpreter.resources,
    )

    # 2. Create a Semantic Kernel agent for the Azure AI agent
    agent = AzureAIAgent(
        client=client,
        definition=agent_definition,
    )

    # 3. Create a thread for the agent
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: AzureAIAgentThread = None

    try:
        print(f"# User: '{TASK}'")
        # 4. Invoke the agent for the specified thread for response
        async for response in agent.invoke(messages=TASK, thread=thread):
            if response.role != AuthorRole.TOOL:
                print(f"# Agent: {response}")
            thread = response.thread
    finally:
        # 6. Cleanup: Delete the thread and agent
        await thread.delete() if thread else None
        await client.agents.delete_agent(agent.id)

# User: 'Use code to determine the values in the Fibonacci sequence that that are less then the value of 101.'
# Agent: def fibonacci_sequence_below_n(max_value):
    fib_sequence = []
    a, b = 0, 1
    while a < max_value:
        fib_sequence.append(a)
        a, b = b, a + b
    return fib_sequence

# Get the Fibonacci sequence values less than 101
fibonacci_below_101 = fibonacci_sequence_below_n(101)
fibonacci_below_101
# Agent: The values in the Fibonacci sequence that are less than 101 are:

\[ 0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89 \]


## 🧪 Case 3.2 open API with Azure AI Agent 
---

In [18]:
USER_INPUTS = [
    "What is the name and population of the country that uses currency with abbreviation KRW",
    "What is the current weather in the capital city of the country?",
]

ai_agent_settings = AzureAIAgentSettings.create()

async with (
    DefaultAzureCredential() as creds,
    AzureAIAgent.create_client(credential=creds) as client,
):
    # 1. Read in the OpenAPI spec files
    openapi_spec_file_path = "resources"
    with open(os.path.join(openapi_spec_file_path, "weather.json")) as weather_file:
        weather_openapi_spec = json.loads(weather_file.read())
    with open(os.path.join(openapi_spec_file_path, "countries.json")) as countries_file:
        countries_openapi_spec = json.loads(countries_file.read())

    # 2. Create OpenAPI tools
    # Note that connection or managed identity auth setup requires additional setup in Azure
    auth = OpenApiAnonymousAuthDetails()
    openapi_weather = OpenApiTool(
        name="get_weather",
        spec=weather_openapi_spec,
        description="Retrieve weather information for a location",
        auth=auth,
    )
    openapi_countries = OpenApiTool(
        name="get_country",
        spec=countries_openapi_spec,
        description="Retrieve country information",
        auth=auth,
    )

    # 3. Create an agent on the Azure AI agent service with the OpenAPI tools
    agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        tools=openapi_weather.definitions + openapi_countries.definitions,
    )

    # 4. Create a Semantic Kernel agent for the Azure AI agent
    agent = AzureAIAgent(
        client=client,
        definition=agent_definition,
    )

    # 5. Create a thread for the agent
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: AzureAIAgentThread = None

    try:
        for user_input in USER_INPUTS:
            print(f"# User: '{user_input}'")
            # 7. Invoke the agent for the specified thread for response
            async for response in agent.invoke(messages=user_input, thread=thread):
                if response.role != AuthorRole.TOOL:
                    print(f"# Agent: {response}")
                thread = response.thread
    finally:
        # 8. Cleanup: Delete the thread and agent
        await client.agents.delete_thread(thread.id)
        await client.agents.delete_agent(agent.id)



# User: 'What is the name and population of the country that uses currency with abbreviation KRW'
# Agent: The currency with the abbreviation KRW is the South Korean Won. The country that uses the South Korean Won is South Korea. As of my last update, the estimated population of South Korea is approximately 52 million people. For the most current and precise population figures, consulting a reliable source such as a government database or a world population review would be ideal.
# User: 'What is the current weather in the capital city of the country?'
# Agent: The current weather in Seoul, the capital of South Korea, is as follows:

- **Temperature:** 12°C (54°F)
- **Feels Like:** 11°C (53°F)
- **Weather Condition:** Clear
- **Humidity:** 58%
- **Wind:** WNW at 8 km/h (5 mph)
- **Visibility:** 10 km
- **Pressure:** 1007 mb

The weather is characteristically clear with a gentle wind from the west-northwest.


## 🧪 Case 3.3 bing search API as plugin with ChatCompletionAgent
---

In [19]:
# Simulate a conversation with the agent
USER_INPUTS = [
    "What is today's weather in South Korea?"
]
from semantic_kernel.connectors.search.bing import BingSearch
from semantic_kernel.functions import KernelArguments, KernelParameterMetadata, KernelPlugin

webplugin = KernelPlugin.from_text_search_with_search(
        BingSearch(api_key=os.getenv("BING_SUBSCRIPTION_KEY")),
        plugin_name="bing",
        description="Search the web for information.",
        parameters=[
            KernelParameterMetadata(
                name="query",
                description="The search query.",
                type="str",
                is_required=True,
                type_object=str,
            ),
            KernelParameterMetadata(
                name="top",
                description="The number of results to return.",
                type="int",
                is_required=False,
                default_value=1,
                type_object=int,
            ),
            KernelParameterMetadata(
                name="skip",
                description="The number of results to skip.",
                type="int",
                is_required=False,
                default_value=0,
                type_object=int,
            ),
            # KernelParameterMetadata(
            #     name="site",
            #     description="The site to search.",
            #     default_value="https://github.com/microsoft/semantic-kernel/tree/main/python",
            #     type="str",
            #     is_required=False,
            #     type_object=str,
            # ),
        ],
    )



agent = ChatCompletionAgent(
        service=AzureChatCompletion(),
        name="Host",
        instructions="Answer questions from web search results. Add the web search reference url to the answer.",
        plugins=[webplugin],
    )

# 2. Create a thread to hold the conversation
# If no thread is provided, a new thread will be
# created and returned with the initial response
thread: ChatHistoryAgentThread = None

for user_input in USER_INPUTS:
    print(f"# User: {user_input}")
    # 4. Invoke the agent for a response
    response = await agent.get_response(messages=user_input, thread=thread)
    print(f"# {response.name}: {response} ")
    thread = response.thread


########################################
# invoke_stream mode 
########################################
# for user_input in USER_INPUTS:
#         print(f"# User: '{user_input}'")
#         first_chunk = True
#         # 4. Invoke the agent for the current message and print the response
#         async for response in agent.invoke_stream(messages=user_input, thread=thread):
#             thread = response.thread
#             if first_chunk:
#                 print(f"# {response.name}: ", end="", flush=True)
#                 first_chunk = False
#             print(response.content, end="", flush=True)
#         print()

await thread.delete() if thread else None

# User: What is today's weather in South Korea?


# Host: Today in South Korea, the weather is mostly clear and pleasant. In Seoul, the current temperature is around 21°C (approximately 69.8°F), with a predicted minimum of 13°C (55.4°F) tonight. The maximum temperature is expected to be around 16°C, and temperatures will drop to about 10°C in the evening.

For more details, you can check the latest updates [here](https://www.weather.com). 


## 🧪 Case 3.4 file search tool with AzureAssistantAgent (Open AI Asisstant)
---

In [20]:

# Simulate a conversation with the agent
USER_INPUTS = {
    "Who is the youngest employee?",
    "Who works in sales?",
    "I have a customer request, who can help me?",
}


# 1. Create the client using Azure OpenAI resources and configuration
client, model = AzureAssistantAgent.setup_resources()

# 2. Read and upload the file to the Azure OpenAI assistant service
pdf_file_path = "resources/employees.pdf"


with open(pdf_file_path, "rb") as file:
    file = await client.files.create(file=file, purpose="assistants")

vector_store = await client.vector_stores.create(
    name="step4_assistant_file_search",
    file_ids=[file.id],
)

# 3. Create file search tool with uploaded resources
file_search_tool, file_search_tool_resources = AzureAssistantAgent.configure_file_search_tool(vector_store.id)

# 4. Create the assistant on the Azure OpenAI service with the file search tool
definition = await client.beta.assistants.create(
    model=model,
    instructions="Find answers to the user's questions in the provided file.",
    name="FileSearch",
    tools=file_search_tool,
    tool_resources=file_search_tool_resources,
)

# 5. Create a Semantic Kernel agent for the Azure OpenAI assistant
agent = AzureAssistantAgent(
    client=client,
    definition=definition,
)

# 6. Create a new thread for use with the assistant
# If no thread is provided, a new thread will be
# created and returned with the initial response
thread: AssistantAgentThread = None

try:
    for user_input in USER_INPUTS:
        print(f"# User: '{user_input}'")
        # 7. Invoke the agent for the current thread and print the response
        async for response in agent.invoke(messages=user_input, thread=thread):
            print(f"# Agent: {response}")
            thread = response.thread
finally:
    # 9. Clean up the resources
    await client.files.delete(file.id)
    await client.vector_stores.delete(vector_store.id)
    await client.beta.threads.delete(thread.id)
    await client.beta.assistants.delete(agent.id)

# User: 'Who works in sales?'
# Agent: In the sales department, the following individuals work:

1. **Mariam Jaslyn** - Sales representative
2. **Angelino Embla** - Sales representative
3. **Hicran Bea** - Sales manager【4:0†source】.
# User: 'Who is the youngest employee?'
# Agent: The youngest employee is **Teodor Britton**, who is an accountant born on January 9, 1997【8:0†source】.
# User: 'I have a customer request, who can help me?'
# Agent: For assistance with a customer request, you can reach out to the following individuals:

1. **Mariam Jaslyn** - Sales representative
2. **Hicran Bea** - Sales manager
3. **Angelino Embla** - Sales representative【12:0†source】.


## 🧪 Case 3.5 BingGrounding with Azure AI Agent (Optional)
---

In [21]:
from azure.ai.projects.models import BingGroundingTool
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
USER_INPUTS = [
    "What is today's weather in South Korea?",
    "What is the new hotels in NYC 2025?",
]

creds = DefaultAzureCredential()

project_client = AIProjectClient.from_connection_string(
    credential=creds,
    conn_str=os.getenv("AZURE_AI_AGENT_PROJECT_CONNECTION_STRING"),
)
bing_connection = project_client.connections.get(
    connection_name=os.getenv("BING_GROUNDING_CONNECTION_NAME"),
)
conn_id = bing_connection.id


# 1. Create an agent with a code interpreter on the Azure AI agent service
bing = BingGroundingTool(connection_id=conn_id)
with project_client:
    agent = project_client.agents.create_agent(
        model=os.getenv("AZURE_AI_AGENT_MODEL_DEPLOYMENT_NAME"),
        name="my-bing-agent",
        instructions="You are a helpful assistant",
        tools=bing.definitions,
        headers={"x-ms-enable-preview": "true"}
    )
    #if you want to use existing agent, you can use the following code
    # agent = project_client.agents.get_agent(agent_id="your agent id")
    
    # Create thread for communication
    thread = project_client.agents.create_thread()
    
    # Create message to thread
    message = project_client.agents.create_message(
        thread_id=thread.id,
        role="user",
        content=USER_INPUTS[0],
    )
    print(f"Created message, ID: {message.id}")

    run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id)
    
    run_steps = project_client.agents.list_run_steps(run_id=run.id, thread_id=thread.id)
    run_steps_data = run_steps['data']
    
    if run.status == "failed":
        print(f"Run failed: {run.last_error}")

    # Delete the assistant when done
    project_client.agents.delete_agent(agent.id)
    
    messages = project_client.agents.list_messages(thread_id=thread.id)
    print(f"Messages: {messages}")
    
    for msg in messages['data']:
        if msg['role'] == 'assistant':
            print(f"Message ID: {msg['id']}")
            print(f"Role: {msg['role']}")
            print(f"Content: {msg['content'][0]['text']['value']}")
            print(f"Created At: {msg['created_at']}")
            print(f"Metadata: {msg['metadata']}")
            
            # Print URL information if available
            annotations = msg['content'][0]['text'].get('annotations', [])
            for annotation in annotations:
                if annotation['type'] == 'url_citation':
                    url_info = annotation['url_citation']
                    print(f"URL: {url_info['url']}")
                    print(f"Title: {url_info['title']}")
            print("-" * 50)

Created message, ID: msg_ruQneTcLMXkt2ZWIjlNwMpKu
Messages: {'object': 'list', 'data': [{'id': 'msg_CSsrrCXmvSEef9FhAntzff43', 'object': 'thread.message', 'created_at': 1744288188, 'assistant_id': 'asst_DL4OpkzoeKQFVq6kdB9Or2Gk', 'thread_id': 'thread_iXWOLWzxf0LL8yJzB8u5KnoV', 'run_id': 'run_RQOvfFGTJmlDUXRPmkLi84Rf', 'role': 'assistant', 'content': [{'type': 'text', 'text': {'value': 'Today, April 10, 2025, in South Korea, the weather in Seoul is experiencing light rain with temperatures ranging from 18°C during the day to 10°C at night【3:0†source】.', 'annotations': [{'type': 'url_citation', 'text': '【3:0†source】', 'start_index': 154, 'end_index': 166, 'url_citation': {'url': 'https://www.weather25.com/asia/south-korea/seoul?page=month&month=April', 'title': 'Seoul weather in April 2025 | Seoul 14 day weather'}}]}}], 'attachments': [], 'metadata': {}}, {'id': 'msg_ruQneTcLMXkt2ZWIjlNwMpKu', 'object': 'thread.message', 'created_at': 1744288186, 'assistant_id': None, 'thread_id': 'threa